Import potrzebnych bibliotek

In [75]:
import pandas as pd
pd.set_option('display.precision', 2)
import numpy as np
from pathlib import Path

Wczyatnie danych z pliku CSV

In [53]:
plik = Path('data', 'diabetes_prediction_dataset.csv')
df = pd.read_csv(plik)

Eksploracyjna analiza danych

Sprawdźmy jakie osoby skończyły z cukrzycą (stworzymy dwa dataframe'y z osobami z cukrzcą i bez oraz porównamy różne zmienne w tych dwóch grupach)

In [58]:
diabetes = df[df['diabetes'] != 0]
no_diabetes = df[df['diabetes'] == 0]

In [73]:
avg_glucose_lvl_diab = pd.to_numeric(diabetes['blood_glucose_level']).mean()
avg_glucose_lvl_no_diab = pd.to_numeric(no_diabetes['blood_glucose_level']).mean()

In [76]:
print(f'Średni poziom glukozy wśród osób bez cukrzycy to: {avg_glucose_lvl_no_diab}, a wśród osób z cukrzycą to: {avg_glucose_lvl_diab}')

Średni poziom glukozy wśród osób bez cukrzycy to: 132.85246994535518, a wśród osób z cukrzycą to: 194.09470588235294


Powyższego wniosku można było się spodziewać. Sprawdźmy teraz ile osób z cukrzycą i bez ma nadciśnienie

In [146]:
procent_nadcisnienia_cukrzyca = pd.DataFrame(diabetes.groupby('hypertension').agg({'diabetes': sum}).reset_index(drop=True))
populacja = procent_nadcisnienia_cukrzyca['diabetes'].sum()
procent_nadcisnienia_cukrzyca['udzial_proc'] = ''
for row in procent_nadcisnienia_cukrzyca.index:
    procent_nadcisnienia_cukrzyca.at[row, 'udzial_proc'] = (procent_nadcisnienia_cukrzyca.at[row, 'diabetes'] / populacja)*100
    
procent_nadcisnienia_cukrzyca['udzial_proc']

0    75.44
1    24.56
Name: udzial_proc, dtype: object

In [145]:
procent_nadcisnienia_nie_cukrzyca = pd.DataFrame(no_diabetes.groupby('hypertension')['diabetes'].count().reset_index(drop=True))
populacja = procent_nadcisnienia_nie_cukrzyca['diabetes'].sum()
procent_nadcisnienia_nie_cukrzyca['udzial_proc'] = ''
for row in procent_nadcisnienia_nie_cukrzyca.index:
    procent_nadcisnienia_nie_cukrzyca.at[row, 'udzial_proc'] = (procent_nadcisnienia_nie_cukrzyca.at[row, 'diabetes'] / populacja)*100
    
procent_nadcisnienia_nie_cukrzyca['udzial_proc']

0    94.1
1     5.9
Name: udzial_proc, dtype: object

Jak widzimy 25% osób z cukrzycą ma nadciśnienie, a tylko 5.9% osób bez cukrzycy ma nadciśnienie.
Sprawdźmy jak wygląda udział historii chorób serca wśród osób z i bez cukrzycy

In [144]:
procent_chorob_serca_nie_cukrzyca = pd.DataFrame(no_diabetes.groupby('heart_disease')['diabetes'].count().reset_index(drop=True))
populacja = procent_chorob_serca_nie_cukrzyca['diabetes'].sum()
procent_chorob_serca_nie_cukrzyca['udzial_proc'] = ''
for row in procent_chorob_serca_nie_cukrzyca.index:
    procent_chorob_serca_nie_cukrzyca.at[row, 'udzial_proc'] = (procent_chorob_serca_nie_cukrzyca.at[row, 'diabetes'] / populacja)*100
    
procent_chorob_serca_nie_cukrzyca['udzial_proc']

0    97.08
1     2.92
Name: udzial_proc, dtype: object

In [143]:
procent_chorob_serca_cukrzyca = pd.DataFrame(diabetes.groupby('heart_disease').agg({'diabetes': sum}).reset_index())
populacja = procent_chorob_serca_cukrzyca['diabetes'].sum()
procent_chorob_serca_cukrzyca['udzial_proc'] = ''
for row in procent_chorob_serca_cukrzyca.index:
    procent_chorob_serca_cukrzyca.at[row, 'udzial_proc'] = (procent_chorob_serca_cukrzyca.at[row, 'diabetes'] / populacja)*100
    
procent_chorob_serca_cukrzyca['udzial_proc']

0    85.09
1    14.91
Name: udzial_proc, dtype: object

Z tego kroku wynika że osoby z cukrzycą mają w prawie 15% populacji historię chorób serca podczas gdy wśród osób bez cukrzycy ten udział to tylko niecałe 3% osób.
Sprawdźmy średnie BMI w obu grupach

In [147]:
bmi_diab = diabetes['bmi'].mean()
bmi_no_diab = no_diabetes['bmi'].mean()
print(f'Średnie BMI wśród osób z cukrzycą to: {bmi_diab}, a wśród osób bez cukrzycy to: {bmi_no_diab}')

Średnie BMI wśród osób z cukrzycą to: 31.988382352941176, a wśród osób bez cukrzycy to: 26.88716349726776


Po raz kolejny okazuje się że osoby z cukrzycą prowadzą mniej zdrowy tryb życia

Sprawdzenie typów danych oraz przekształcenie zmiennych kategorycznych

In [54]:
df.dtypes

gender                  object
age                    float64
hypertension             int64
heart_disease            int64
smoking_history         object
bmi                    float64
HbA1c_level            float64
blood_glucose_level      int64
diabetes                 int64
dtype: object

Wiek podany jest w pełnych latach więc zmieniamy typ danych na int

In [41]:
df['age'] = df['age'].astype(int)

Pozostałe dane które wymagają zmiany typu danych to zmienne kategoryczne - postanowiłem wykorzystać metodę one-hot encoding, w której mając kilka wartości kategorycznych stworzę tyle kolumn binarnych ile mam możliwości wartości - w tym przpadku stworzę 6 column z których tylko jedna przybierze wartość 1 a pozostałe 0 - w ten sposób przedstawię wartość kategoryczną binarnie 

In [42]:
df['smoking_history'].drop_duplicates().reset_index(drop=True)

0          never
1        No Info
2        current
3         former
4           ever
5    not current
Name: smoking_history, dtype: object

Jak widać muszę stworzyć 6 kolumn

In [43]:
df['smoking_history_never'] = np.where(df['smoking_history'] == 'never',  1, 0)
df['smoking_history_no_info'] = np.where(df['smoking_history'] == 'No Info',  1, 0)
df['smoking_history_current'] = np.where(df['smoking_history'] == 'current',  1, 0)
df['smoking_history_former'] = np.where(df['smoking_history'] == 'former',  1, 0)
df['smoking_history_ever'] = np.where(df['smoking_history'] == 'ever',  1, 0)
df['smoking_history_not_current'] = np.where(df['smoking_history'] == 'not current',  1, 0)

Teraz należy zmienić typ danych zmiennej gender

In [44]:
df['gender'].drop_duplicates().reset_index(drop=True)

0    Female
1      Male
2     Other
Name: gender, dtype: object

Jak widać mamy trzy możliwości wartości zmiennej gender - również zastosuję one-hot encoding

In [45]:
df['is_male'] = np.where(df['gender'] == 'Male', 1, 0)
df['is_female'] = np.where(df['gender'] == 'Female', 1, 0)
df['is_other'] = np.where(df['gender'] == 'Other', 1, 0)

Możemy sprawdzić skuteczność konwersji danych poprzez zrobienie testowej kolumny z sumą z kolumn konwertowanych (wynik musi zawsze wynosić 1)

In [46]:
df['smoking_test'] = df['smoking_history_current'] + df['smoking_history_former'] + df['smoking_history_ever'] + df['smoking_history_never'] + df['smoking_history_no_info'] + df['smoking_history_not_current']

df['gender_test'] = df['is_male'] + df['is_female'] + df['is_other']

In [47]:
df['smoking_test'].drop_duplicates().reset_index(drop=True)

0    1
Name: smoking_test, dtype: int32

In [48]:
df['gender_test'].drop_duplicates().reset_index(drop=True)

0    1
Name: gender_test, dtype: int32

Jak widać nie mamy wartości różnych od 1 więc poprawnie zastosowaliśmy one-hot encoding

Sprawdźmy typy danych oraz zróbmy drop niepotrzebynch już kolumn

In [50]:
df.drop(columns=['gender', 'smoking_history', 'smoking_test', 'gender_test'], inplace=True)

In [51]:
df.dtypes

age                              int32
hypertension                     int64
heart_disease                    int64
bmi                            float64
HbA1c_level                    float64
blood_glucose_level              int64
diabetes                         int64
smoking_history_never            int32
smoking_history_no_info          int32
smoking_history_current          int32
smoking_history_former           int32
smoking_history_ever             int32
smoking_history_not_current      int32
is_male                          int32
is_female                        int32
is_other                         int32
dtype: object

Następnym krokiem będzie standaryzacja danych